# Figures in supplementary

Håkon Wiik Ånes (hakon.w.anes@ntnu.no), 2022-11-29, Norwegian University of Science and Technology (NTNU)

Some figures in the supplementary are generated in the `figures_in_paper` notebook.

In [1]:
# Switch to interactive Matplotlib backend (e.g. qt5) for control point determination
%matplotlib qt5

from datetime import date
import importlib_metadata
import os

import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import scipy.ndimage as ndi
import numpy as np
import skimage.exposure as ske
import skimage.color as skc
import skimage.transform as skt


# Interactive plotting OFF
plt.ioff()

# Other Matplotlib parameters
plt.rcParams.update({"font.size": 11, "figure.dpi": 300})
scalebar_kw = dict(
    units="um",
    dimension="si-length",
    location="lower left",
    pad=0.2,
    sep=3,
    border_pad=0.5,
    box_alpha=0.6,
    fixed_value=15,
)
savefig_kw = dict(bbox_inches="tight", pad_inches=0.01, transparent=False, dpi=300)
title_kw = dict(ha="left", va="bottom", fontweight="bold")

# Other parameters
dir_article = "/home/hakon/phd/artikler_abstract/202204_aanes_p_texture_almn_alloy_actamat/latex/fig"

print("Run date: ", date.today())
print("\nSoftware versions\n-----------------")
for pkg in ["matplotlib", "numpy", "scikit-image"]:
    if pkg == "numpy":
        ver = np.__version__
    else:
        ver = importlib_metadata.version(pkg)
    print(pkg, ":", ver)

Run date:  2022-11-30

Software versions
-----------------
matplotlib : 3.6.2
numpy : 1.23.5
scikit-image : 0.19.3


In [2]:
step_size = dict(ebsd=100, bse=1 / 39.2e-3)
SCALE_EBSD_BSE = step_size["ebsd"] / step_size["bse"]  # Global variable used inside functions

In [3]:
# Padding used for BSE images when EBSD map extends outside BSE images
ebsd_pad = {
    "0s": {1: [(0, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(0, 0), (0, 0)]},
    "175c": {1: [(100, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(200, 100), (100, 0)]},
    "300c": {1: [(0, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(0, 0), (0, 0)]},
    "325c": {1: [(0, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(100, 0), (0, 0)]},
}

In [4]:
def cut_to_roi(image: np.ndarray) -> np.ndarray:
    """Remove black parts outside of an image.

    Parameters
    ----------
    image
        Image.
    
    Returns
    -------
    image_out
        Image of different size with black parts cut.
    """
    if image.ndim == 3:
        mask = skc.rgb2gray(image[..., :3]) != 0
    else:
        mask = image != 0
    mask = ndi.binary_fill_holes(mask)
    y, x = np.where(mask)
    ymin, ymax = np.min(y), np.max(y)
    xmin, xmax = np.min(x), np.max(x)
    image_out = image[ymin:ymax, xmin:xmax]
    return image_out

In [5]:
def get_bse_image(dir_data: str) -> np.ndarray:
    bse = plt.imread(os.path.join(dir_data, "imreg/reg_bse_no_padding.png"))
    bse = skc.rgb2gray(bse[..., :3])

    bse = cut_to_roi(bse)

    # Blank out black inside ROI
    mask = bse != 0
    mask = ndi.binary_fill_holes(mask)
    bse[~mask] = np.nan
    
    return bse

In [6]:
def get_bse_particle_image(dir_data: str) -> np.ndarray:
    bse_part = plt.imread(os.path.join(dir_data, "partdet/bse_particles_roi.png"))

    # Upscale to enable use of EBSD ROI mask from image registration notebook
    bse = plt.imread(os.path.join(dir_data, "bse/4500x_cropped2_fused_cropped_cropped.png"))
    bse_part = skt.rescale(bse_part, tuple(np.array(bse.shape) / np.array(bse_part.shape[:2])) + (1,))

    # Read mask
    mask = np.load(os.path.join(dir_data, "imreg/mask_ebsd_correct.npy"))
    mask = skt.rescale(mask, SCALE_EBSD_BSE)

    # Pad particle BSE image or crop mask and EBSD ROI, depending on which have the
    # greater shape
    mask_bse_shape_diff = np.array(mask.shape) - np.array(bse_part.shape[:2])
    if mask_bse_shape_diff[0] > 0:
        # Add rows to the end of the BSE image
        bse_part2 = np.pad(bse_part, [(0, mask_bse_shape_diff[0]), (0, 0), (0, 0)])
    else:
        # Add rows to the end of the rescaled mask and EBSD intensity map
        mask = mask[:bse_part.shape[0]]
        mask = np.pad(mask, [(0, -mask_bse_shape_diff[0]), (0, 0)])
        bse_part2 = bse_part.copy()
    if mask_bse_shape_diff[1] > 0:
        # Add columns to the end of the BSE image
        bse_part2 = np.pad(bse_part2, [(0, 0), (0, mask_bse_shape_diff[1]), (0, 0)])
    else:
        # Add columns to the end of the rescaled mask and EBSD intensity map
        mask = mask[:, :bse_part.shape[1]]
        mask = np.pad(mask, [(0, 0), (0, -mask_bse_shape_diff[1])])
    
    # Apply mask to BSE particle image
    bse_part2[~mask] = 0
    
    # Remove black borders
    bse_part3 = cut_to_roi(bse_part2)
    
    # Crop to fit above BSE image
    bse_part3 = bse_part3[:bse.shape[0], :bse.shape[1]]
    
    return bse_part3

Plot combined figures for all datasets at a certain condition

In [10]:
sample = "325c"  # 0s, 175c, 300c, 325c

for i in range(1, 4):
    dir_data = os.path.join("/home/hakon/phd/data/p/prover", sample, str(i))
    
    bse = get_bse_image(dir_data)
    bse_part = get_bse_particle_image(dir_data)
    
    # Pad Both BSE images to fit the EBSD ROI exactly
    pad = np.array(ebsd_pad[sample][i]) / SCALE_EBSD_BSE
    pad = np.round(pad).astype(int)
    bse2 = np.pad(bse, pad, constant_values=np.nan)
    bse_part2 = np.pad(bse_part, list(pad) + [(0, 0)])

    # Read inverse pole figure (IPF) map with orientations colored according to
    # the lattice vector parallel to the rolling direction (RD) upwards
    ideal_map = plt.imread(os.path.join(dir_data, "mtex/maps_grains_ideal_particles.png"))
    ideal_map = ideal_map[5:-5, 5:-5]
    
    # Combine images in a figure with a common scalebar
    figsize = (9 * bse2.shape[1] / max(bse2.shape), 3 * bse2.shape[0] / max(bse2.shape))
    fig, axes = plt.subplots(ncols=3, figsize=figsize)
    axes[0].imshow(bse2, cmap="gray")
    axes[1].imshow(bse_part2)
    axes[2].imshow(ideal_map)
    axes[0].add_artist(ScaleBar(dx=step_size["bse"] * 1e-3, **scalebar_kw))
    for ax in axes:
        ax.axis("off")
    fig.tight_layout(pad=0.3)
    fig.savefig(os.path.join(dir_article, "maps", f"maps_{sample}_{i}.png"), **savefig_kw)